In [1]:
import sqlite3
import duckdb
import pandas as pd

### Conexão e Teste

In [2]:
sqlite_con = sqlite3.connect('data/northwind.db')

df_customers = pd.read_sql_query("SELECT * FROM Customers LIMIT 5", sqlite_con)

sqlite_con.close()

print("\nExemplo de dados da tabela 'Customers':")

print(df_customers)


Exemplo de dados da tabela 'Customers':
  CustomerID                         CompanyName         ContactName  \
0      ALFKI                 Alfreds Futterkiste        Maria Anders   
1      ANATR  Ana Trujillo Emparedados y helados        Ana Trujillo   
2      ANTON             Antonio Moreno Taquería      Antonio Moreno   
3      AROUT                     Around the Horn        Thomas Hardy   
4      BERGS                  Berglunds snabbköp  Christina Berglund   

           ContactTitle                        Address         City  \
0  Sales Representative                  Obere Str. 57       Berlin   
1                 Owner  Avda. de la Constitución 2222  México D.F.   
2                 Owner                Mataderos  2312  México D.F.   
3  Sales Representative                120 Hanover Sq.       London   
4   Order Administrator                Berguvsvägen  8        Luleå   

            Region PostalCode  Country           Phone             Fax  
0   Western Europe      12

### Criar Tabelas Duckdb em memória

In [3]:
duckdb_con = duckdb.connect(database=':memory:')

duckdb_con.execute("CREATE TABLE Customers AS SELECT * FROM sqlite_scan('data/northwind.db', 'Customers')")
duckdb_con.execute("CREATE TABLE Employees AS SELECT * FROM sqlite_scan('data/northwind.db', 'Employees')")
duckdb_con.execute("CREATE TABLE Orders AS SELECT * FROM sqlite_scan('data/northwind.db', 'Orders')")
duckdb_con.execute("CREATE TABLE OrderDetails AS SELECT * FROM sqlite_scan('data/northwind.db', 'Order Details')")
duckdb_con.execute("CREATE TABLE Products AS SELECT * FROM sqlite_scan('data/northwind.db', 'Products')")
duckdb_con.execute("CREATE TABLE Categories AS SELECT * FROM sqlite_scan('data/northwind.db', 'Categories')")

### Análises

In [4]:
# 1. Contagem de clientes por país
conta_clientes = duckdb_con.sql("""
    SELECT Country, COUNT(*) AS CustomerCount
    FROM Customers
    GROUP BY Country
    ORDER BY CustomerCount DESC
""")

print("1. Contagem de clientes por país:")

print(conta_clientes)

1. Contagem de clientes por país:
┌─────────────┬───────────────┐
│   Country   │ CustomerCount │
│   varchar   │     int64     │
├─────────────┼───────────────┤
│ USA         │            13 │
│ France      │            11 │
│ Germany     │            11 │
│ Brazil      │             9 │
│ UK          │             7 │
│ Spain       │             5 │
│ Mexico      │             5 │
│ Venezuela   │             4 │
│ Italy       │             3 │
│ Canada      │             3 │
│ Argentina   │             3 │
│ Denmark     │             2 │
│ Portugal    │             2 │
│ NULL        │             2 │
│ Finland     │             2 │
│ Switzerland │             2 │
│ Austria     │             2 │
│ Belgium     │             2 │
│ Sweden      │             2 │
│ Poland      │             1 │
│ Ireland     │             1 │
│ Norway      │             1 │
├─────────────┴───────────────┤
│ 22 rows           2 columns │
└─────────────────────────────┘



In [5]:
# 2. Contagem de funcionários por Cargo
funcionario_titulo = duckdb_con.sql("""
    SELECT Title, COUNT(*) AS EmployeeCount
    FROM Employees
    GROUP BY Title
    ORDER BY EmployeeCount DESC
""")

print("\n2. Contagem de funcionários por título:")

print(funcionario_titulo)


2. Contagem de funcionários por título:
┌──────────────────────────┬───────────────┐
│          Title           │ EmployeeCount │
│         varchar          │     int64     │
├──────────────────────────┼───────────────┤
│ Sales Representative     │             6 │
│ Sales Manager            │             1 │
│ Vice President, Sales    │             1 │
│ Inside Sales Coordinator │             1 │
└──────────────────────────┴───────────────┘



In [ ]:
# 3. Receita total por cliente
receita_total_cliente = duckdb_con.sql("""
    SELECT C.CustomerID, C.CompanyName, SUM(OD.UnitPrice * OD.Quantity * (1 - OD.Discount)) AS TotalRevenue
    FROM Orders O
    JOIN OrderDetails OD ON O.OrderID = OD.OrderID
    JOIN Customers C ON O.CustomerID = C.CustomerID
    GROUP BY C.CustomerID, C.CompanyName
    ORDER BY TotalRevenue DESC
""")

print("\n3. Receita total por cliente:")

print(receita_total_cliente)